In [1]:
# main.py

from config import *
from utils.seed import set_seed
from data.splits import create_splits, create_splits_from_file
from data.transforms import get_transforms
from data.datamodule import ImageDataModule
from models.efficientnet import EfficientNetLit

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from torch.utils.data import WeightedRandomSampler

import torch
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

from utils.make_sampler import make_sampler


def main():
    set_seed(SEED)

    all_fold_metrics = []

    for fold in range(1, N_FOLDS + 1):
        print(f"\n{'='*60}")
        print(f" START FOLD {fold}/{N_FOLDS}")
        print(f"{'='*60}\n")

        train_df, val_df, test_df, classes = create_splits_from_file(
            split_csv=SPLIT_FILE,
            images_dir=METADATA_DIR,
            fold=fold,
        )


    #train_df, val_df, test_df, classes = create_splits(
    #    metadata_csv= METADATA_DIR,
    #    images_dir= METADATA_DIR,
    #    seed=SEED,
    #    test_size=TEST_SIZE,
    #    val_size=VAL_SIZE,
    #)

        print(
            len(train_df),
            len(val_df),
            len(test_df),
            train_df.lesion_id.nunique(),
            val_df.lesion_id.nunique(),
            test_df.lesion_id.nunique(),
        )
    
        print("TRAIN class distribution:")
        print(train_df["label"].value_counts().sort_index())
        
        train_sampler = make_sampler(train_df["label"].values)
    
        train_tfms, val_tfms = get_transforms()
    
        datamodule = ImageDataModule(
            train_df=train_df,
            val_df=val_df,
            test_df=test_df,
            train_tfms=train_tfms,
            val_tfms=val_tfms,
            batch_size=BATCH_SIZE,
            num_workers=NUM_WORKERS,
            train_sampler=train_sampler,
         )
    
        
        model = EfficientNetLit(
            num_classes=len(classes),
            lr=LR,
            weight_decay=WEIGHT_DECAY,
        )
    
    
        logger = CSVLogger("logs", name="efficientnet", version=f"fold_{fold}",)
    
    
        early_stop = pl.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=20,
            mode="min",
            verbose=True,
        )
    
    
        trainer = pl.Trainer(
            max_epochs=EPOCHS,
            accelerator="gpu",
            devices=1,
            logger=logger,
            callbacks=[
                ModelCheckpoint(
                        monitor="val_loss",
                        mode="min",
                        filename=f"fold{fold}" + "-{epoch}-{val_loss:.4f}",
                    ),
                early_stop,
            ],
        )
    
        trainer.fit(model, datamodule=datamodule)

        test_metrics = trainer.test(
            model,
            datamodule=datamodule,
            ckpt_path="best",
        )
        
        all_fold_metrics.append(test_metrics[0])

    print("\n CROSS-VALIDATION RESULTS")
    for i, m in enumerate(all_fold_metrics, 1):
        print(f"Fold {i}: {m}")

    # среднее
    mean_bal_acc = np.mean(
        [m["test_balanced_acc"] for m in all_fold_metrics]
    )
    print(f"\n MEAN BALANCED ACC: {mean_bal_acc:.4f}")


if __name__ == "__main__":
    main()


Seed set to 42



🚀 START FOLD 1/5

⚠️ NOTE: 700 lesions shared between TRAIN and VAL (image-level validation by design)
12991 867 3467 3897 749 987
TRAIN class distribution:
label
0    4236
1    3494
2     412
3    1155
4    2758
5     801
6     135
Name: count, dtype: int64


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name       ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ encoder    │ EfficientNet     │  4.0 M │ train │     0 │
│ 1 │ classifier │ Sequential       │  824 K │ train │     0 │
│ 2 │ criterion  │ CrossEntropyLoss │      0 │ train │     0 │
└───┴────────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 4.8 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.8 M                                                                                                
Total estimated model params size (MB): 19                                                                         
Modules in train mode: 344                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

...

Trainable params: 4.8 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 4.8 M                                                                                                
Total estimated model params size (MB): 19                                                                         
Modules in train mode: 344                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

Metric val_loss improved. New best score: 1.158
Metric val_loss improved by 0.200 >= min_delta = 0.0. New best score: 0.959
Metric val_loss improved by 0.070 >= min_delta = 0.0. New best score: 0.889
Metric val_loss improved by 0.074 >= min_delta = 0.0. New best score: 0.815
Metric val_loss improved by 0.025 >= min_delta = 0.0. New best score: 0.790
Metric val_loss improved by 0.044 >= min_delta = 0.0. New best score: 0.746
Metric val_loss improved by 0.049 >= min_delta = 0.0. New best score: 0.697
Metric val_loss improved by 0.041 >= min_delta = 0.0. New best score: 0.655
Metric val_loss improved by 0.030 >= min_delta = 0.0. New best score: 0.625
Metric val_loss improved by 0.030 >= min_delta = 0.0. New best score: 0.595
Metric val_loss improved by 0.031 >= min_delta = 0.0. New best score: 0.564

Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

/home/.conda/envs/env/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
